In [ ]:
import get_price.get_futures_price as gfp
panel_data = gfp.get_futures_price_preloaded(ticker_head='CL')
unique_dates = panel_data['settle_date'].unique()

data4day = panel_data[panel_data['settle_date']==unique_dates[4000]]


In [7]:
import get_price.curve_data as cd
rolling_data = cd.get_rolling_curve_data(ticker_head='CL',num_contracts=15,
                                         front_tr_dte_limit=10,
                                        date_from=20050101,
                                        date_to=20160112)



In [8]:
rolling_data[9]['change5']

settle_date
2005-01-03    0.86
2005-01-04    1.95
2005-01-05    1.67
2005-01-06    2.34
2005-01-07    2.84
2005-01-10    2.05
2005-01-11    1.28
2005-01-12    1.13
2005-01-13    1.02
2005-01-14    1.66
2005-01-18    1.61
2005-01-19    1.86
2005-01-20   -0.43
2005-01-21    0.16
2005-01-24   -1.19
2005-01-25   -1.07
2005-01-26   -1.11
2005-01-27    0.03
2005-01-28   -1.47
2005-01-31   -0.90
2005-02-01   -0.59
2005-02-02    0.49
2005-02-03    0.31
2005-02-04    1.09
2005-02-07    0.79
2005-02-08    1.72
2005-02-09    0.49
2005-02-10    1.40
2005-02-11    3.09
2005-02-14    3.17
              ... 
2015-11-30   -3.30
2015-12-01   -2.26
2015-12-02   -2.80
2015-12-03   -3.06
2015-12-04   -1.66
2015-12-07   -0.64
2015-12-08   -1.70
2015-12-09   -2.26
2015-12-10   -1.53
2015-12-11   -1.69
2015-12-14   -2.61
2015-12-15   -0.42
2015-12-16    0.76
2015-12-17    0.04
2015-12-18    1.48
2015-12-21    0.76
2015-12-22    0.66
2015-12-23   -0.38
2015-12-24    0.17
2015-12-28   -2.26
2015-12-29   -1.94


In [16]:
import pandas as pd
wuhu = [rolling_data[x]['close_price']/rolling_data[x+1]['close_price'] for x in range(len(rolling_data)-1)]
merged_data = pd.concat(wuhu,axis=1)
yield_data = merged_data.values

wuhu = [rolling_data[x]['change5']-rolling_data[x+1]['change5'] for x in range(len(rolling_data)-1)]
merged_data = pd.concat(wuhu,axis=1)
change5_data = merged_data.values

wuhu = [rolling_data[x]['tr_dte'] for x in range(len(rolling_data)-1)]
merged_data = pd.concat(wuhu,axis=1)
tr_dte_data = merged_data.values

wuhu = [rolling_data[x]['cont_indx'] for x in range(len(rolling_data)-1)]
merged_data = pd.concat(wuhu,axis=1)
cont_indx_data = merged_data.values

[rolling_data[x]['ticker'][-1] for x in range(len(rolling_data)-1)]

['CLH2016',
 'CLJ2016',
 'CLK2016',
 'CLM2016',
 'CLN2016',
 'CLQ2016',
 'CLU2016',
 'CLV2016',
 'CLX2016',
 'CLZ2016',
 'CLF2017',
 'CLG2017',
 'CLH2017',
 'CLJ2017']

In [3]:
import shared.statistics as stats
pca_out = stats.get_pca(data_input=yield_data,n_components=2)

In [6]:
pca_out['loadings'][0]

array([-0.49997407, -0.39189241, -0.33474993, -0.29856778, -0.26485688,
       -0.23865464, -0.22332526, -0.20779813, -0.19726683, -0.18491655,
       -0.17505439, -0.16208256, -0.15373119, -0.14639053])

In [4]:
residuals = yield_data-pca_out['model_fit']
residuals

array([[ -5.06704273e-04,  -6.97115779e-04,   1.99864099e-03, ...,
          3.06464154e-04,  -1.24318238e-04,  -1.26665189e-03],
       [ -5.88076800e-04,  -5.49326133e-04,   2.07911238e-03, ...,
          2.44387560e-04,  -1.35138469e-04,  -1.20840694e-03],
       [ -1.18433507e-03,   2.16608861e-04,   2.44655483e-03, ...,
          1.21139126e-05,  -3.62385122e-04,  -1.44063444e-03],
       ..., 
       [  1.79731953e-03,  -2.30505417e-03,  -2.31112645e-03, ...,
         -7.46861781e-05,  -1.68515247e-04,   2.95409993e-04],
       [  2.12929683e-03,  -2.82153122e-03,  -2.54582012e-03, ...,
         -2.10959333e-04,  -6.68255993e-05,   4.09485593e-04],
       [  2.15207177e-03,  -2.77614077e-03,  -2.54480947e-03, ...,
          8.86315952e-05,  -6.37745027e-07,   5.02255942e-04]])

In [5]:
import backtesting.backtest_curve_pca as bcp

pnl_path = []

unique_dates = rolling_data[0].index

for i in range(len(unique_dates)):
    
    pnl_path.append(bcp.backtest_curve_pca_4date(residuals=residuals[i,:],
                                                 change_data=change5_data[i,:],
                                                 tr_dte_data = tr_dte_data[i,:]))







In [17]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca_fit = pca.fit(yield_data)
pca_fit.

In [7]:

import matplotlib.pyplot as plt
import numpy as np
plt.plot(np.cumsum(pnl_path))
plt.grid()               
plt.show()

In [6]:
import opportunity_constructs.curve_pca as cpc
import pandas as pd
#report_out = cpc.get_curve_pca_report(ticker_head='CL', date_to=20160112 )

report_out['factor_load1']







0    -0.491369
1    -0.383422
2    -0.325928
3    -0.289781
4    -0.256626
5    -0.231031
6    -0.216446
7    -0.201348
8    -0.191452
9    -0.178841
10   -0.169455
11   -0.157283
12   -0.149513
13   -0.142330
14   -0.134478
15   -0.128106
16   -0.124586
Name: factor_load1, dtype: float64

In [13]:
np.mean(pnl_path)

0.017107258938244906

In [ ]:
%pylab inline
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
plt.rcParams['legend.fontsize'] = 10
ax.plot(class1_sample[0,:], class1_sample[1,:], class1_sample[2,:],
        'o', markersize=8, color='blue', alpha=0.5, label='class1')
ax.plot(class2_sample[0,:], class2_sample[1,:], class2_sample[2,:],
        '^', markersize=8, alpha=0.5, color='red', label='class2')

plt.title('Samples for class 1 and class 2')
ax.legend(loc='upper right')

plt.show()

In [ ]:
all_samples = np.concatenate((class1_sample, class2_sample), axis=1)
assert all_samples.shape ==(3,40), "The matrix has not the dimensions 3x40"

In [ ]:
mean_x = np.mean(all_samples[0,:])
mean_y = np.mean(all_samples[1,:])
mean_z = np.mean(all_samples[2,:])

mean_vector = np.array([[mean_x],[mean_y],[mean_z]])
print('Mean Vector:\n', mean_vector)






In [ ]:
scatter_matrix = np.zeros((3,3))

for i in range(all_samples.shape[1]):
    scatter_matrix +=(all_samples[:,i].reshape(3,1)).dot(
    (all_samples[:,i].reshape(3,1)-mean_vector).T)
print('Scatter Matrix\n',scatter_matrix)    

In [ ]:
cov_mat = np.cov([all_samples[0,:], all_samples[1,:],all_samples[2,:]])
print('Covariance Matrix:\n', cov_mat)

In [ ]:
# eigenvectors and eigenvalues for the from the scatter matrix
eig_val_sc, eig_vec_sc = np.linalg.eig(scatter_matrix)

# eigenvectors and eigenvalues for the from the covariance matrix
eig_val_cov, eig_vec_cov = np.linalg.eig(cov_mat)

for i in range(len(eig_val_sc)):
    eigvec_sc = eig_vec_sc[:,i].reshape(1,3).T
    eigvec_cov = eig_vec_cov[:,i].reshape(1,3).T
    assert eigvec_sc.all() == eigvec_cov.all(), 'Eigenvectors are not identical'

    print('Eigenvector {}: \n{}'.format(i+1, eigvec_sc))
    print('Eigenvalue {} from scatter matrix: {}'.format(i+1, eig_val_sc[i]))
    print('Eigenvalue {} from covariance matrix: {}'.format(i+1, eig_val_cov[i]))
    print('Scaling factor: ', eig_val_sc[i]/eig_val_cov[i])
    print(40 * '-')

In [ ]:
for i in range(len(eig_val_sc)):
    eigv = eig_vec_sc[:,i].reshape(1,3).T
    np.testing.assert_array_almost_equal(scatter_matrix.dot(eigv),
                                         eig_val_sc[i] * eigv,
                                         decimal=6, err_msg='', verbose=True)

In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from matplotlib.patches import FancyArrowPatch


class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        FancyArrowPatch.draw(self, renderer)

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection='3d')

ax.plot(all_samples[0,:], all_samples[1,:], all_samples[2,:],
        'o', markersize=8, color='green', alpha=0.2)
ax.plot([mean_x], [mean_y], [mean_z],
        'o', markersize=10, color='red', alpha=0.5)
for v in eig_vec_sc.T:
    a = Arrow3D([mean_x, v[0]+mean_x],
                [mean_y, v[1]+mean_y],
                [mean_z, v[2]+mean_z],
                mutation_scale=20, lw=3, arrowstyle="-|>", color="r")

    ax.add_artist(a)
ax.set_xlabel('x_values')
ax.set_ylabel('y_values')
ax.set_zlabel('z_values')

plt.title('Eigenvectors')

plt.show()
multi_columns = pd.MultiIndex.from_product([[ticker1, ticker2, ticker3],data_1.columns.values], names=['first', 'second'])


In [ ]:
transformed = matrix_w.T.dot(all_samples)
assert transformed.shape == (2,40), "The matrix is not 2x40 dimensional."
transformed
matrix_w

In [ ]:
import matplotlib.pyplot as plt
plt.plot(merged_dataframe.index,(merged_dataframe['BOH2016']['close_price']+merged_dataframe['BON2016']['close_price'])/2-
         merged_dataframe['BOK2016']['close_price'])
         
         
plt.show()

In [ ]:
(merged_dataframe['BOH2016']['close_price']+merged_dataframe['BON2016']['close_price'])/2-merged_dataframe['BOK2016']['close_price']

In [ ]:


import matplotlib.pyplot as plt
plt.plot(range(len(aligned_dataframe.index)),aligned_dataframe['c1']['tr_dte'],
         range(len(aligned_dataframe.index)),aligned_dataframe['c2']['tr_dte'],
         range(len(aligned_dataframe.index)),aligned_dataframe['c3']['tr_dte'])
plt.grid()         
         
plt.show()

    

    

In [ ]:
merged_dataframe['c1']['tr_dte'].isnull().sum()
merged_dataframe['c3']['tr_dte'].isnull().sum()

In [ ]:
futures_dataframe

In [ ]:
print(trDte1)
print(trDte2)
print(trDte3)

In [ ]:
bf_price = (aligned_dataframe['c1']['close_price']+aligned_dataframe['c3']['close_price'])/2-aligned_dataframe['c2']['close_price']

In [ ]:
plt.plot(range(len(aligned_dataframe.index)),bf_price)
plt.grid()         
         
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(price_ratio)
plt.grid()
plt.show()

In [5]:
import numpy as np
nan_matrix = np.empty((2,2))
nan_matrix[:] = np.NAN
nan_matrix

array([[ nan,  nan],
       [ nan,  nan]])